In [1]:
from sqlalchemy import create_engine
import pandas as pd
from dotenv import load_dotenv
import os

load_dotenv()  # take environment variables from .env.
database_url = os.getenv("DATABASE_URL")
engine = create_engine(database_url) 

In [2]:

query = "SELECT * FROM sebastien.title_basics LIMIT 10000;"
with engine.connect() as conn, conn.begin():
    df = pd.read_sql_query(query, engine)

df.head()

,tconst,titletype,primarytitle,originaltitle,isadult,startyear,endyear,runtimeminutes,genres,averagerating,numvotes
0,tt32349431,tvEpisode,Comment distinguer la critique du harcèlement?...,Comment distinguer la critique du harcèlement?...,False,2024.0,NaN,NaN,Documentary,NaN,NaN
1,tt32349433,tvEpisode,Episode #1.7,Episode #1.7,False,2024.0,NaN,NaN,"Drama,Romance",NaN,NaN
2,tt32349436,tvEpisode,Episode #1.8,Episode #1.8,False,2024.0,NaN,NaN,"Drama,Romance",NaN,NaN
3,tt32349437,tvEpisode,The Uber Rush,The Uber Rush,False,2024.0,NaN,NaN,Comedy,NaN,NaN
4,tt32349438,tvEpisode,Episode #1.9,Episode #1.9,False,2024.0,NaN,NaN,"Drama,Romance",NaN,NaN


In [4]:
df.drop(columns=["originaltitle"], inplace=True)
df

,tconst,titletype,primarytitle,isadult,startyear,endyear,runtimeminutes,genres,averagerating,numvotes
0,tt32349431,tvEpisode,Comment distinguer la critique du harcèlement?...,False,2024.0,NaN,NaN,Documentary,NaN,NaN
1,tt32349433,tvEpisode,Episode #1.7,False,2024.0,NaN,NaN,"Drama,Romance",NaN,NaN
2,tt32349436,tvEpisode,Episode #1.8,False,2024.0,NaN,NaN,"Drama,Romance",NaN,NaN
3,tt32349437,tvEpisode,The Uber Rush,False,2024.0,NaN,NaN,Comedy,NaN,NaN
4,tt32349438,tvEpisode,Episode #1.9,False,2024.0,NaN,NaN,"Drama,Romance",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
9995,tt32378907,tvEpisode,Faszination Wasser: Flussdeltas - Paradiese zw...,False,2024.0,NaN,44.0,"Documentary,History",NaN,NaN
9996,tt32378912,tvEpisode,Faszination Wasser: Wasserfälle - Die Urgewalt...,False,2024.0,NaN,44.0,"Documentary,History",NaN,NaN
9997,tt32378919,tvEpisode,Episode #5.71,False,1998.0,NaN,NaN,Game-Show,NaN,NaN
9998,tt3237892,tvEpisode,Episode #1.6689,False,2013.0,NaN,18.0,"Drama,Romance",NaN,NaN


In [10]:
query = "SELECT * FROM sebastien.title_episode;"
with engine.connect() as conn, conn.begin():
    df_episode = pd.read_sql_query(query, engine)

df_episode.head()

,tconst,parenttconst,seasonnumber,episodenumber
0,tt12155488,tt12145510,1.0,7.0
1,tt1215549,tt0990536,1.0,3.0
2,tt12155490,tt12092480,1.0,211.0
3,tt12155492,tt12092480,1.0,212.0
4,tt12155494,tt12145510,1.0,8.0


In [11]:
df_merge = df.merge(df_episode, on="tconst", how="left")
df_merge

,tconst,titletype,primarytitle,isadult,startyear,endyear,runtimeminutes,genres,averagerating,numvotes,parenttconst,seasonnumber,episodenumber
0,tt32349431,tvEpisode,Comment distinguer la critique du harcèlement?...,False,2024.0,NaN,NaN,Documentary,NaN,NaN,tt8690468,NaN,NaN
1,tt32349433,tvEpisode,Episode #1.7,False,2024.0,NaN,NaN,"Drama,Romance",NaN,NaN,tt32325609,1.0,7.0
2,tt32349436,tvEpisode,Episode #1.8,False,2024.0,NaN,NaN,"Drama,Romance",NaN,NaN,tt32325609,1.0,8.0
3,tt32349437,tvEpisode,The Uber Rush,False,2024.0,NaN,NaN,Comedy,NaN,NaN,tt30946404,1.0,25.0
4,tt32349438,tvEpisode,Episode #1.9,False,2024.0,NaN,NaN,"Drama,Romance",NaN,NaN,tt32325609,1.0,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,tt32378907,tvEpisode,Faszination Wasser: Flussdeltas - Paradiese zw...,False,2024.0,NaN,44.0,"Documentary,History",NaN,NaN,tt0382491,NaN,NaN
9996,tt32378912,tvEpisode,Faszination Wasser: Wasserfälle - Die Urgewalt...,False,2024.0,NaN,44.0,"Documentary,History",NaN,NaN,tt0382491,NaN,NaN
9997,tt32378919,tvEpisode,Episode #5.71,False,1998.0,NaN,NaN,Game-Show,NaN,NaN,tt0233098,5.0,71.0
9998,tt3237892,tvEpisode,Episode #1.6689,False,2013.0,NaN,18.0,"Drama,Romance",NaN,NaN,tt0092325,1.0,6689.0


In [12]:
df_merge_parent = df.merge(df_episode, left_on="tconst", right_on="parenttconst", how="left", suffixes=('', '_parent'))
df_merge_parent

,tconst,titletype,primarytitle,isadult,startyear,endyear,runtimeminutes,genres,averagerating,numvotes,tconst_parent,parenttconst,seasonnumber,episodenumber
0,tt32349431,tvEpisode,Comment distinguer la critique du harcèlement?...,False,2024.0,NaN,NaN,Documentary,NaN,NaN,NaN,NaN,NaN,NaN
1,tt32349433,tvEpisode,Episode #1.7,False,2024.0,NaN,NaN,"Drama,Romance",NaN,NaN,NaN,NaN,NaN,NaN
2,tt32349436,tvEpisode,Episode #1.8,False,2024.0,NaN,NaN,"Drama,Romance",NaN,NaN,NaN,NaN,NaN,NaN
3,tt32349437,tvEpisode,The Uber Rush,False,2024.0,NaN,NaN,Comedy,NaN,NaN,NaN,NaN,NaN,NaN
4,tt32349438,tvEpisode,Episode #1.9,False,2024.0,NaN,NaN,"Drama,Romance",NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12146,tt32378907,tvEpisode,Faszination Wasser: Flussdeltas - Paradiese zw...,False,2024.0,NaN,44.0,"Documentary,History",NaN,NaN,NaN,NaN,NaN,NaN
12147,tt32378912,tvEpisode,Faszination Wasser: Wasserfälle - Die Urgewalt...,False,2024.0,NaN,44.0,"Documentary,History",NaN,NaN,NaN,NaN,NaN,NaN
12148,tt32378919,tvEpisode,Episode #5.71,False,1998.0,NaN,NaN,Game-Show,NaN,NaN,NaN,NaN,NaN,NaN
12149,tt3237892,tvEpisode,Episode #1.6689,False,2013.0,NaN,18.0,"Drama,Romance",NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
combined = pd.concat([df_merge, df_merge_parent], ignore_index=True)
combined

,tconst,titletype,primarytitle,isadult,startyear,endyear,runtimeminutes,genres,averagerating,numvotes,parenttconst,seasonnumber,episodenumber,tconst_parent
0,tt32349431,tvEpisode,Comment distinguer la critique du harcèlement?...,False,2024.0,NaN,NaN,Documentary,NaN,NaN,tt8690468,NaN,NaN,NaN
1,tt32349433,tvEpisode,Episode #1.7,False,2024.0,NaN,NaN,"Drama,Romance",NaN,NaN,tt32325609,1.0,7.0,NaN
2,tt32349436,tvEpisode,Episode #1.8,False,2024.0,NaN,NaN,"Drama,Romance",NaN,NaN,tt32325609,1.0,8.0,NaN
3,tt32349437,tvEpisode,The Uber Rush,False,2024.0,NaN,NaN,Comedy,NaN,NaN,tt30946404,1.0,25.0,NaN
4,tt32349438,tvEpisode,Episode #1.9,False,2024.0,NaN,NaN,"Drama,Romance",NaN,NaN,tt32325609,1.0,9.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22146,tt32378907,tvEpisode,Faszination Wasser: Flussdeltas - Paradiese zw...,False,2024.0,NaN,44.0,"Documentary,History",NaN,NaN,NaN,NaN,NaN,NaN
22147,tt32378912,tvEpisode,Faszination Wasser: Wasserfälle - Die Urgewalt...,False,2024.0,NaN,44.0,"Documentary,History",NaN,NaN,NaN,NaN,NaN,NaN
22148,tt32378919,tvEpisode,Episode #5.71,False,1998.0,NaN,NaN,Game-Show,NaN,NaN,NaN,NaN,NaN,NaN
22149,tt3237892,tvEpisode,Episode #1.6689,False,2013.0,NaN,18.0,"Drama,Romance",NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
combined[combined["tconst"].duplicated()]

,tconst,titletype,primarytitle,isadult,startyear,endyear,runtimeminutes,genres,averagerating,numvotes,parenttconst,seasonnumber,episodenumber,tconst_parent
10000,tt32349431,tvEpisode,Comment distinguer la critique du harcèlement?...,False,2024.0,NaN,NaN,Documentary,NaN,NaN,NaN,NaN,NaN,NaN
10001,tt32349433,tvEpisode,Episode #1.7,False,2024.0,NaN,NaN,"Drama,Romance",NaN,NaN,NaN,NaN,NaN,NaN
10002,tt32349436,tvEpisode,Episode #1.8,False,2024.0,NaN,NaN,"Drama,Romance",NaN,NaN,NaN,NaN,NaN,NaN
10003,tt32349437,tvEpisode,The Uber Rush,False,2024.0,NaN,NaN,Comedy,NaN,NaN,NaN,NaN,NaN,NaN
10004,tt32349438,tvEpisode,Episode #1.9,False,2024.0,NaN,NaN,"Drama,Romance",NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22146,tt32378907,tvEpisode,Faszination Wasser: Flussdeltas - Paradiese zw...,False,2024.0,NaN,44.0,"Documentary,History",NaN,NaN,NaN,NaN,NaN,NaN
22147,tt32378912,tvEpisode,Faszination Wasser: Wasserfälle - Die Urgewalt...,False,2024.0,NaN,44.0,"Documentary,History",NaN,NaN,NaN,NaN,NaN,NaN
22148,tt32378919,tvEpisode,Episode #5.71,False,1998.0,NaN,NaN,Game-Show,NaN,NaN,NaN,NaN,NaN,NaN
22149,tt3237892,tvEpisode,Episode #1.6689,False,2013.0,NaN,18.0,"Drama,Romance",NaN,NaN,NaN,NaN,NaN,NaN
